# Ensemble of ensembles - model stacking

* **Ensemble with different types of classifiers**: 
  * Different types of classifiers (E.g., logistic regression, decision trees, random forest, etc.) are fitted on the same training data
  * Results are combined based on either 
    * majority voting (classification) or 
    * average (regression)
  

* **Ensemble with a single type of classifier**: 
  * Bootstrap samples are drawn from training data 
  * With each bootstrap sample, model (E.g., Individual model may be decision trees, random forest, etc.) will be fitted 
  * All the results are combined to create an ensemble. 
  * Suitabe for highly flexible models that is prone to overfitting / high variance. 

***

## Combining Method

* **Majority voting or average**: 
  * Classification: Largest number of votes (mode) 
  * Regression problems: Average (mean).
  
  
* **Method of application of meta-classifiers on outcomes**: 
  * Binary outcomes: 0 / 1 from individual classifiers
  * Meta-classifier is applied on top of the individual classifiers. 
  
  
* **Method of application of meta-classifiers on probabilities**: 
  * Probabilities are obtained from individual classifiers. 
  * Applying meta-classifier
  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.pop('EmployeeNumber')
df.pop('Over18')
df.pop('StandardHours')
df.pop('EmployeeCount')
y = df['Attrition']
X = df
X.pop('Attrition')
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()
y = le.fit_transform(y)
ind_BusinessTravel = pd.get_dummies(df['BusinessTravel'], prefix='BusinessTravel')
ind_Department = pd.get_dummies(df['Department'], prefix='Department')
ind_EducationField = pd.get_dummies(df['EducationField'], prefix='EducationField')
ind_Gender = pd.get_dummies(df['Gender'], prefix='Gender')
ind_JobRole = pd.get_dummies(df['JobRole'], prefix='JobRole')
ind_MaritalStatus = pd.get_dummies(df['MaritalStatus'], prefix='MaritalStatus')
ind_OverTime = pd.get_dummies(df['OverTime'], prefix='OverTime')
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime])
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime, df.select_dtypes(['int64'])], axis=1)
df1.dropna(inplace=True)
df1.shape

(1470, 51)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, y,
                                                    test_size=0.2)

In [4]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    '''
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))    

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))      
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))
        

## Model 1: Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [8]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=True)
print_score(tree_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       995
           1       1.00      1.00      1.00       181

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[995   0]
 [  0 181]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.7585

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.89      0.86       238
           1       0.31      0.21      0.25        56

    accuracy                           0.76       294
   macro avg       0.57      0.55      0.55       294
weighted avg       0.73      0.76      0.74       294


Confusion Matrix: 
 [[211  27]
 [ 44  12]]

ROC AUC: 0.5504



## Model 2: Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train.ravel())

RandomForestClassifier()

In [11]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       995
           1       1.00      1.00      1.00       181

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[995   0]
 [  0 181]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.8367

Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.99      0.91       238
           1       0.83      0.18      0.29        56

    accuracy                           0.84       294
   macro avg       0.84      0.59      0.60       294
weighted avg       0.84      0.84      0.79       294


Confusion Matrix: 
 [[236   2]
 [ 46  10]]

ROC AUC: 0.5851



In [12]:
en_en = pd.DataFrame()

In [13]:
tree_clf.predict_proba(X_train)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [14]:
tree_clf.predict_proba?

In [15]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [16]:
en_en.head()

,tree_clf,rf_clf,0
0,0.0,0.05,0
1,0.0,0.00,0
2,0.0,0.05,0
3,0.0,0.03,0
4,0.0,0.05,0


In [17]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

In [18]:
en_en.head()

,tree_clf,rf_clf,ind
0,0.0,0.05,0
1,0.0,0.00,0
2,0.0,0.05,0
3,0.0,0.03,0
4,0.0,0.05,0


# Meta Classifier

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
m_clf = LogisticRegression(fit_intercept=False)

In [21]:
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

LogisticRegression(fit_intercept=False)

In [22]:
en_test = pd.DataFrame()

In [23]:
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

In [24]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

In [25]:
tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [26]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [27]:
en_test.columns = tmp

In [28]:
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined    0   1
ind              
0         211  27
1          44  12


In [29]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.7585


In [30]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       238
           1       0.31      0.21      0.25        56

    accuracy                           0.76       294
   macro avg       0.57      0.55      0.55       294
weighted avg       0.73      0.76      0.74       294



***

# Single Classifier

In [31]:
df = pd.read_csv("data\WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [32]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [33]:
df.Attrition.value_counts() / df.Attrition.count()

No     0.838776
Yes    0.161224
Name: Attrition, dtype: float64

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
from sklearn.ensemble import BaggingClassifier

In [36]:
from sklearn.ensemble import AdaBoostClassifier

In [37]:
class_weight = {0:0.834, 1:0.166}

In [38]:
pd.Series(list(y_train)).value_counts() / pd.Series(list(y_train)).count()

TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


[0]    0.846088
[1]    0.153912
dtype: float64

In [39]:
forest = RandomForestClassifier(class_weight=class_weight)

In [40]:
ada = AdaBoostClassifier(base_estimator=forest, n_estimators=100,
                         learning_rate=0.5, random_state=42)

In [41]:
ada.fit(X_train, y_train.ravel())

AdaBoostClassifier(base_estimator=RandomForestClassifier(class_weight={0: 0.834,
                                                                       1: 0.166}),
                   learning_rate=0.5, n_estimators=100, random_state=42)

In [42]:
print_score(ada, X_train, X_test, y_train, y_test, train=True)
print_score(ada, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       995
           1       1.00      1.00      1.00       181

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176


Confusion Matrix: 
 [[995   0]
 [  0 181]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.8333

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.99      0.91       238
           1       0.82      0.16      0.27        56

    accuracy                           0.83       294
   macro avg       0.83      0.58      0.59       294
weighted avg       0.83      0.83      0.78       294


Confusion Matrix: 
 [[236   2]
 [ 47   9]]

ROC AUC: 0.5762



In [43]:
bag_clf = BaggingClassifier(base_estimator=ada, n_estimators=50,
                            max_samples=1.0, max_features=1.0, bootstrap=True,
                            bootstrap_features=False, n_jobs=-1,
                            random_state=42)

In [44]:
bag_clf.fit(X_train, y_train.ravel())

BaggingClassifier(base_estimator=AdaBoostClassifier(base_estimator=RandomForestClassifier(class_weight={0: 0.834,
                                                                                                        1: 0.166}),
                                                    learning_rate=0.5,
                                                    n_estimators=100,
                                                    random_state=42),
                  n_estimators=50, n_jobs=-1, random_state=42)

In [45]:
print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 0.9685

Classification Report: 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       995
           1       1.00      0.80      0.89       181

    accuracy                           0.97      1176
   macro avg       0.98      0.90      0.93      1176
weighted avg       0.97      0.97      0.97      1176


Confusion Matrix: 
 [[995   0]
 [ 37 144]]

ROC AUC: 0.8978

Test Result:

accuracy score: 0.8231

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.99      0.90       238
           1       0.75      0.11      0.19        56

    accuracy                           0.82       294
   macro avg       0.79      0.55      0.54       294
weighted avg       0.81      0.82      0.76       294


Confusion Matrix: 
 [[236   2]
 [ 50   6]]

ROC AUC: 0.5494



***